In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import random
import pickle

In [2]:
df = pd.read_csv("features.csv")

In [3]:
df

,video_id,frame,Nose_x,Nose_y,Nose_prob,Neck_x,Neck_y,Neck_prob,RShoulder_x,RShoulder_y,...,LHeel_prob,RBigToe_x,RBigToe_y,RBigToe_prob,RSmallToe_x,RSmallToe_y,RSmallToe_prob,RHeel_x,RHeel_y,RHeel_prob
0,13577910,13577910-processed_000000000058_keypoints.json,151.934,55.2863,0.908112,134.935,97.0895,0.838085,99.7177,97.0471,...,0.732763,87.964,393.365,0.703311,87.9081,389.455,0.643066,128.3830,380.315,0.729301
1,13577910,13577910-processed_000000000210_keypoints.json,214.583,37.1114,0.890615,171.434,82.7045,0.697061,170.1870,86.6004,...,0.661197,244.528,449.470,0.733939,234.1400,453.339,0.688678,183.2520,442.970,0.662346
2,13577910,13577910-processed_000000000205_keypoints.json,221.091,37.1081,0.868994,171.479,76.2160,0.679368,171.5290,77.4813,...,0.284568,154.516,446.893,0.483251,149.2710,448.162,0.480059,111.4510,419.451,0.556726
3,13577910,13577910-processed_000000000459_keypoints.json,180.594,56.5680,0.899521,140.108,98.4054,0.720787,138.8220,101.0070,...,0.786555,172.800,445.554,0.736735,164.9680,450.782,0.661102,120.5770,442.979,0.837428
4,13577910,13577910-processed_000000000604_keypoints.json,141.441,46.2081,0.917255,181.936,95.7544,0.798888,167.5530,90.5475,...,0.724951,214.505,433.823,0.414325,214.5920,429.911,0.295466,243.2590,415.546,0.732996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080765,14751302,14751302-processed_000000000640_keypoints.json,0.000,0.0000,0.000000,0.000,0.0000,0.000000,0.0000,0.0000,...,0.764837,146.678,204.098,0.455000,153.2230,205.400,0.466335,150.5580,213.233,0.747068
2080766,14751302,14751302-processed_000000000184_keypoints.json,0.000,0.0000,0.000000,134.924,10.9351,0.775681,150.5250,12.2118,...,0.769522,150.619,133.640,0.678411,149.3100,136.257,0.707545,138.8560,132.368,0.742950
2080767,14751302,14751302-processed_000000000191_keypoints.json,120.543,3.7283,0.234840,138.831,14.8999,0.917227,149.3060,14.8787,...,0.793793,146.686,127.113,0.356382,150.6310,125.783,0.357174,149.2910,117.962,0.560404
2080768,14751302,14751302-processed_000000000590_keypoints.json,0.000,0.0000,0.000000,0.000,0.0000,0.000000,0.0000,0.0000,...,0.237002,124.484,369.898,0.212867,0.0000,0.000,0.000000,140.1170,359.422,0.316340


In [4]:
body_parts = [ "Nose","Neck","RShoulder","RElbow","RWrist","LShoulder","LElbow","LWrist","MidHip","RHip","RKnee","RAnkle","LHip","LKnee","LAnkle","REye","LEye","REar","LEar","LBigToe","LSmallToe","LHeel","RBigToe","RSmallToe","RHeel"]
all_features = ['video_id', 'frame'] + [f"{body_part}_{elem}" for body_part in body_parts for elem in ['x', 'y', 'prob']]

### Original Dataset

In [5]:
used_body_parts = ["LAnkle", "RAnkle", "LKnee", "RKnee", "LHip", "RHip", "LBigToe", "RBigToe"]
used_features = [f"{body_part}_{elem}" for body_part in used_body_parts for elem in ['x', 'y']]

In [6]:
df_subset = df[['video_id', 'frame'] + used_features]
df_subset

,video_id,frame,LAnkle_x,LAnkle_y,RAnkle_x,RAnkle_y,LKnee_x,LKnee_y,RKnee_x,RKnee_y,LHip_x,LHip_y,RHip_x,RHip_y,LBigToe_x,LBigToe_y,RBigToe_x,RBigToe_y
0,13577910,13577910-processed_000000000058_keypoints.json,161.0740,381.586,119.2610,371.198,141.506,287.6010,102.312,290.2620,150.575,193.6420,110.125,193.6640,129.733,398.568,87.964,393.365
1,13577910,13577910-processed_000000000210_keypoints.json,102.3250,416.831,194.9490,432.545,163.676,321.5590,221.065,317.6380,162.390,215.8640,175.418,218.4580,147.993,444.279,244.528,449.470
2,13577910,13577910-processed_000000000205_keypoints.json,151.9100,423.375,129.6880,406.432,171.430,324.1780,189.763,321.5730,183.219,228.8850,170.146,228.9090,194.986,440.363,154.516,446.893
3,13577910,13577910-processed_000000000459_keypoints.json,78.8179,407.702,129.7380,435.104,150.589,329.4190,154.523,330.7010,151.918,240.6710,144.077,243.2550,108.832,440.335,172.800,445.554
4,13577910,13577910-processed_000000000604_keypoints.json,200.1660,436.417,232.8370,412.893,172.737,329.3900,181.943,329.3990,181.898,235.4030,162.386,234.1360,161.008,453.408,214.505,433.823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080765,14751302,14751302-processed_000000000640_keypoints.json,116.6720,211.951,150.5930,204.087,110.138,128.4100,151.843,123.1890,108.795,44.8888,150.604,43.5828,120.585,213.237,146.678,204.098
2080766,14751302,14751302-processed_000000000184_keypoints.json,131.0890,128.381,141.4570,129.743,136.248,89.3029,144.056,93.1811,130.987,57.8933,147.959,57.8899,140.181,131.029,150.619,133.640
2080767,14751302,14751302-processed_000000000191_keypoints.json,141.4070,121.904,149.3090,112.755,138.840,89.2359,150.529,86.6497,133.606,55.2908,150.532,54.0206,140.121,130.997,146.686,127.113
2080768,14751302,14751302-processed_000000000590_keypoints.json,205.4410,401.201,129.7120,346.364,200.180,191.0640,119.231,180.6030,196.293,16.1673,110.123,14.8815,202.762,433.797,124.484,369.898


In [7]:
video_lengths = df_subset.groupby("video_id").count()['frame']

In [8]:
video_lengths_filtered = video_lengths[video_lengths >= 500]

In [9]:
filtered_video_ids = video_lengths_filtered.index.tolist()

In [10]:
filtered_df_subset = df_subset[df_subset['video_id'].isin(filtered_video_ids)]
filtered_df_subset

,video_id,frame,LAnkle_x,LAnkle_y,RAnkle_x,RAnkle_y,LKnee_x,LKnee_y,RKnee_x,RKnee_y,LHip_x,LHip_y,RHip_x,RHip_y,LBigToe_x,LBigToe_y,RBigToe_x,RBigToe_y
0,13577910,13577910-processed_000000000058_keypoints.json,161.0740,381.586,119.2610,371.198,141.506,287.6010,102.312,290.2620,150.575,193.6420,110.125,193.6640,129.733,398.568,87.964,393.365
1,13577910,13577910-processed_000000000210_keypoints.json,102.3250,416.831,194.9490,432.545,163.676,321.5590,221.065,317.6380,162.390,215.8640,175.418,218.4580,147.993,444.279,244.528,449.470
2,13577910,13577910-processed_000000000205_keypoints.json,151.9100,423.375,129.6880,406.432,171.430,324.1780,189.763,321.5730,183.219,228.8850,170.146,228.9090,194.986,440.363,154.516,446.893
3,13577910,13577910-processed_000000000459_keypoints.json,78.8179,407.702,129.7380,435.104,150.589,329.4190,154.523,330.7010,151.918,240.6710,144.077,243.2550,108.832,440.335,172.800,445.554
4,13577910,13577910-processed_000000000604_keypoints.json,200.1660,436.417,232.8370,412.893,172.737,329.3900,181.943,329.3990,181.898,235.4030,162.386,234.1360,161.008,453.408,214.505,433.823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080765,14751302,14751302-processed_000000000640_keypoints.json,116.6720,211.951,150.5930,204.087,110.138,128.4100,151.843,123.1890,108.795,44.8888,150.604,43.5828,120.585,213.237,146.678,204.098
2080766,14751302,14751302-processed_000000000184_keypoints.json,131.0890,128.381,141.4570,129.743,136.248,89.3029,144.056,93.1811,130.987,57.8933,147.959,57.8899,140.181,131.029,150.619,133.640
2080767,14751302,14751302-processed_000000000191_keypoints.json,141.4070,121.904,149.3090,112.755,138.840,89.2359,150.529,86.6497,133.606,55.2908,150.532,54.0206,140.121,130.997,146.686,127.113
2080768,14751302,14751302-processed_000000000590_keypoints.json,205.4410,401.201,129.7120,346.364,200.180,191.0640,119.231,180.6030,196.293,16.1673,110.123,14.8815,202.762,433.797,124.484,369.898


In [89]:
filtered_df_subset.to_csv("original_500.csv", index=False)

In [11]:
df = filtered_df_subset

In [12]:
df.loc[:, "frame_nr"] = df.loc[:, "frame"].apply(lambda x: int(x.split("_")[1]))
df

/var/folders/00/ft6n0snn3g5gtsxj0z9n8lhh0000gp/T/ipykernel_16188/238603174.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "frame_nr"] = df.loc[:, "frame"].apply(lambda x: int(x.split("_")[1]))


,video_id,frame,LAnkle_x,LAnkle_y,RAnkle_x,RAnkle_y,LKnee_x,LKnee_y,RKnee_x,RKnee_y,LHip_x,LHip_y,RHip_x,RHip_y,LBigToe_x,LBigToe_y,RBigToe_x,RBigToe_y,frame_nr
0,13577910,13577910-processed_000000000058_keypoints.json,161.0740,381.586,119.2610,371.198,141.506,287.6010,102.312,290.2620,150.575,193.6420,110.125,193.6640,129.733,398.568,87.964,393.365,58
1,13577910,13577910-processed_000000000210_keypoints.json,102.3250,416.831,194.9490,432.545,163.676,321.5590,221.065,317.6380,162.390,215.8640,175.418,218.4580,147.993,444.279,244.528,449.470,210
2,13577910,13577910-processed_000000000205_keypoints.json,151.9100,423.375,129.6880,406.432,171.430,324.1780,189.763,321.5730,183.219,228.8850,170.146,228.9090,194.986,440.363,154.516,446.893,205
3,13577910,13577910-processed_000000000459_keypoints.json,78.8179,407.702,129.7380,435.104,150.589,329.4190,154.523,330.7010,151.918,240.6710,144.077,243.2550,108.832,440.335,172.800,445.554,459
4,13577910,13577910-processed_000000000604_keypoints.json,200.1660,436.417,232.8370,412.893,172.737,329.3900,181.943,329.3990,181.898,235.4030,162.386,234.1360,161.008,453.408,214.505,433.823,604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2080765,14751302,14751302-processed_000000000640_keypoints.json,116.6720,211.951,150.5930,204.087,110.138,128.4100,151.843,123.1890,108.795,44.8888,150.604,43.5828,120.585,213.237,146.678,204.098,640
2080766,14751302,14751302-processed_000000000184_keypoints.json,131.0890,128.381,141.4570,129.743,136.248,89.3029,144.056,93.1811,130.987,57.8933,147.959,57.8899,140.181,131.029,150.619,133.640,184
2080767,14751302,14751302-processed_000000000191_keypoints.json,141.4070,121.904,149.3090,112.755,138.840,89.2359,150.529,86.6497,133.606,55.2908,150.532,54.0206,140.121,130.997,146.686,127.113,191
2080768,14751302,14751302-processed_000000000590_keypoints.json,205.4410,401.201,129.7120,346.364,200.180,191.0640,119.231,180.6030,196.293,16.1673,110.123,14.8815,202.762,433.797,124.484,369.898,590


In [17]:
video_ids = list(set(df['video_id'].tolist()))
len(video_ids)

2914

In [45]:
video_X = []
SHIFT_BY = 100
for video_id in tqdm(video_ids):
    curr_df = df[df['video_id'] == video_id].sort_values(by='frame_nr')
    nr_frames = len(curr_df)
    for i in range(0, nr_frames-500, SHIFT_BY):
        curr_chunk = curr_df.iloc[i:i+500, :]
        x = curr_chunk[used_features].to_numpy()
        if len(x) != 500:
            print(video_id)
        video_X.append([str(video_id), x])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2914/2914 [00:04<00:00, 614.05it/s]


In [46]:
len(video_X)

6613

In [47]:
with open(f"../data/X_{SHIFT_BY}.pkl", 'wb') as file:
    pickle.dump(video_X, file)

### Match predicted values to features

In [258]:
video_id_predicted_values = pd.read_csv("video_id_predicted_values.csv")
# video_id_predicted_values['video_id'] = pd.to_numeric(video_id_predicted_values['video_id'], downcast='signed', errors='coerce')

In [291]:
filtered_predicted_values = video_id_predicted_values[video_id_predicted_values['video_id'].isin(video_ids)]
filtered_predicted_values

,video_id,side,KneeFlex_maxExtension,speed,cadence,steplen
26,10312701,R,23.962333,0.915005,0.994667,0.394269
27,10312701,L,34.529000,0.897019,0.959000,0.512130
50,10772801,R,43.406333,0.115560,0.839667,-0.069110
51,10772801,L,47.368000,0.139068,0.826333,0.206712
60,10906202,L,29.531369,0.140882,0.278923,0.331527
...,...,...,...,...,...,...
11865,16828501,L,7.087655,0.299313,0.625468,0.208293
11868,18371401,R,9.921491,1.351350,1.090158,0.602977
11869,18371401,L,9.046386,1.343939,1.101355,0.637646
11894,17286301,R,7.365353,1.143967,1.159751,0.499088


In [206]:
common_video_ids_float = filtered_predicted_values.groupby('video_id').count().index.tolist()

In [213]:
common_video_ids_str = [str(int(v_id)) for v_id in common_video_ids_float]

In [217]:
video_ids = common_video_ids_str

In [223]:
random.shuffle(video_ids)

# Calculate the split sizes
total_videos = len(video_ids)
train_size = int(0.8 * total_videos)
val_size = int(0.1 * total_videos)
test_size = total_videos - train_size - val_size  # Ensuring all videos are included

# Split the list
train_videos = video_ids[:train_size]
val_videos = video_ids[train_size:train_size + val_size]
test_videos = video_ids[train_size + val_size:]

# Print the splits
print(f'Total videos: {total_videos}')
print(f'Train videos {len(train_videos)}')
print(f'Validation videos {len(val_videos)}')
print(f'Test videos {len(test_videos)}')

Total videos: 2914
Train videos 2331
Validation videos 291
Test videos 292


In [224]:
split = {"train": train_videos,
        "validation": val_videos,
        "test": test_videos}

In [225]:
with open(f"../data/split.pkl", 'wb') as file:
    pickle.dump(split, file)

In [261]:
video_ids = [str(v_id) for v_id in video_ids]

In [295]:
filtered_predicted_values.to_csv("y.csv", index=False)

In [267]:
filtered_predicted_values

,video_id,side,KneeFlex_maxExtension,speed,cadence,steplen
26,10312701,R,23.962333,0.915005,0.994667,0.394269
27,10312701,L,34.529000,0.897019,0.959000,0.512130
50,10772801,R,43.406333,0.115560,0.839667,-0.069110
51,10772801,L,47.368000,0.139068,0.826333,0.206712
60,10906202,L,29.531369,0.140882,0.278923,0.331527
...,...,...,...,...,...,...
11865,16828501,L,7.087655,0.299313,0.625468,0.208293
11868,18371401,R,9.921491,1.351350,1.090158,0.602977
11869,18371401,L,9.046386,1.343939,1.101355,0.637646
11894,17286301,R,7.365353,1.143967,1.159751,0.499088


In [271]:
filtered_predicted_values[filtered_predicted_values["video_id"] == '10312701']['speed'].tolist()

[0.915005, 0.897019]

In [273]:
filt = filtered_predicted_values.groupby('video_id').count()

In [284]:
good_video_ids = filt[filt['side'] == 2].index.tolist()

In [285]:
len(filtered_predicted_values)

5773

In [292]:
filtered_predicted_values = filtered_predicted_values[filtered_predicted_values['video_id'].isin(good_video_ids)]

In [293]:
len(filtered_predicted_values)

5718

In [294]:
filtered_predicted_values

,video_id,side,KneeFlex_maxExtension,speed,cadence,steplen
26,10312701,R,23.962333,0.915005,0.994667,0.394269
27,10312701,L,34.529000,0.897019,0.959000,0.512130
50,10772801,R,43.406333,0.115560,0.839667,-0.069110
51,10772801,L,47.368000,0.139068,0.826333,0.206712
97,11221801,R,9.374398,1.033810,1.024544,0.542080
...,...,...,...,...,...,...
11865,16828501,L,7.087655,0.299313,0.625468,0.208293
11868,18371401,R,9.921491,1.351350,1.090158,0.602977
11869,18371401,L,9.046386,1.343939,1.101355,0.637646
11894,17286301,R,7.365353,1.143967,1.159751,0.499088
